# Scrapping the data

In [1]:
!pip install lyricsgenius

In [2]:
from lyricsgenius import Genius

In [3]:
genius = Genius('A4acQipb4pwRRNdOq06t4A_CYoYBDJR3rCONvPcN84bEwvia3jS8sVq7LnccAfYc',remove_section_headers = True,)
jul = genius.search_artist("jul", sort="title",get_full_info=True,max_songs=250)
print(jul.songs)

Searching for songs by jul...

Changing artist name to 'JuL'
Song 1: "13*"
Song 2: "135 City"
Song 3: "1er dans la bataille"
Song 4: "1 fois mais pas 2"
"2 visages" is not valid. Skipping.
Song 5: "4 heures du mat’"
Song 6: "4 Juin 2022"
Song 7: "500 Chevaux"
Song 8: "6.35"
Song 9: "Abandonner*"
Song 10: "À bout de nerf"
Song 11: "A.C.C.C"
Song 12: "À coup de crick"
Song 13: "À coup de taser"
Song 14: "À croire que"
Song 15: "Affaire de famille*"
"À FORCE DE VIVRE" is not valid. Skipping.
Song 16: "Africa Twin"
Song 17: "Aghju Capitu"
Song 18: "Agis ou ferme tgl"
Song 19: "Agression*"
Song 20: "Ah c’est comme ça !"
Song 21: "Ailleurs"
Song 22: "Akha*"
Song 23: "À la cité"
Song 24: "À la fou"
Song 25: "À la russe"
Song 26: "À la vida à la muerte"
Song 27: "À la vue"
Song 28: "Alcantara"
Song 29: "Alcoolisé"
Song 30: "À l’impro"
Song 31: "All Eyez on Me"
Song 32: "Allez le sang"
Song 33: "Alors la zone"
Song 34: "Amigo"
Song 35: "Amis ennemis"
Song 36: "Amnésia"
Song 37: "Amore"
Song 38:

# Processing of the data

In [70]:
n=len(jul.songs)
data = []
for i in range(n):
    data.append(jul.songs[i].lyrics)
data = data[1:]

In [197]:
print(data[2])

9 Contributors1 fois mais pas 2 Lyrics

Y'avait pas beaucoup de monde quand j'me noyais
Mais t'façon rien à foutre des autres
J'ai dit à maman : "Tu paieras plus l'loyer"
Tes fils, c'est devenu des hommes
Tu me l'as fait à moi ? 3enenni
Fais pas le con, fais pas le fou, fais demi
Elle sent le Dior, elle veut le sac, L.V
J'ai pas le temps si j'vends chaque ennemi
J'ai passe à bloc en bolide, ouais, ouais
Elle est bonne comme ma weed, ouais, ouais
Ellе m'a souri, j'fais demi, ouais, ouais
Je m'arrête, j'la validе, ouais, ouais

On se plaît, on se voit
Toi et moi on va boire un verra, j'regarde pas l'heure
Tu m'auras, tu m'auras
Tu m'auras une fois mais pas deux, j'fais plus l'erreur

Habibi-bibi
Habibi-bibi-bibi
Habibi-bibi
Habibi-bibi-bibi
You might also like
Ouais ce soir, ça s'est mis frais (Ah-ah, ah-ah)
T'es pas rentré, qu'est-ce t'as fait ? (Ah-ah, ah-ah)
Ouais ce son, c'est pour les vrais (Ah-ah, ah-ah)
Ceux et celles qu'ont galéré (Ah-ah, ah-ah)

J'piste à droite, à gauche
J'peux

We get rid of the first line of the data, which is the header.


In [72]:
cleaned_data = []
for i in data:
    intro_index = i.find("\n")
    cleaned_data.append(i[intro_index+1:])


In [73]:
print(cleaned_data[0])


1.3.5
1.3.5
Eh yo eh yo eh yo ouais c’est du rap
Anti-BDH (1.3.5, 1.3.5)

Rap anti-putes, si tu m’tacles on s’dispute
Rap anti-flics, faut qu’je marque on petit but
J’oublierai pas les heures de geôles l’école, et toutes ces heures de colle
Vas-y rigole, fais pas le mac et lève ta main de mon épaule
Les folles elles veulent la drogue accompagnée du Cherry Cola
Combien y’a d’frère en tole poto à cause du cher Nicolas ?
Ola, c’est un braquage au mic’ mec mec
Ça boit la vodka sec calé, freestyle pét’ au bec
Impecc’ demain c’est vendredi faut qu’j’touche ma paye
J’rentre chez moi il est une heure j’vais encore faire un son qui paye
Assez parlé des femmes “elles te veulent pas” m’a dit ma mère
J’te jure la vie d’ta mère chez moi c’est made in la merde
Ton pote était trop speed j'étais obligé d’le calmer
J’ai trop vu Blow, Scarface pour ça qu’j’aime pas les camés
Au tieks ça vend la beuh venez j’rabats les clients
Quoi qu’est-c’qu’il y a ? Ils m’contrôlent j’les piste en riant
You might als

We create a corpus of the data.

In [74]:
corpus = []
for i in range(len(cleaned_data)):
    lines = cleaned_data[i].split("\n")
    for line in lines:
        line = line.lower()
        line_words = [word for word in line.split(' ') if word.strip() != '']
        line_words.append(' \n ')
        corpus = corpus + line_words
    


In [75]:
len(corpus)

149330

In [240]:
corpus[0:20]

[' \n ',
 '1.3.5',
 ' \n ',
 '1.3.5',
 ' \n ',
 'eh',
 'yo',
 'eh',
 'yo',
 'eh',
 'yo',
 'ouais',
 'c’est',
 'du',
 'rap',
 ' \n ',
 'anti-bdh',
 '(1.3.5,',
 '1.3.5)',
 ' \n ']

In [77]:
words = set(corpus)
nb_words = len(words)

Now we are going to create the predictors/target from the corpus, to do so we use a sliding window of 15 words, and we predict the next word.

In [78]:
sequence_length = 15
step = 2
prev_words = []
next_words = []
sequence = []
for i in range(0, len(corpus) - sequence_length, step):
    prev_words.append(corpus[i: i + sequence_length ])
    next_words.append(corpus[i + sequence_length ])
    


In [241]:
len(prev_words)

74658

# Now we are going to use a LSTM architecture that is going to be trained from scratch.


Imports for the model

In [79]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout,Bidirectional
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np
import ker


In [80]:
# set of words with index as value
word_index = dict((c, i) for i, c in enumerate(words) )
# set of words with index as key
index_word = dict((i, c) for i, c in enumerate(words))

Setting up a generator to feed the model with data and have no memory issues.

In [82]:
# function to generate data from our data set in order to feed our model
def generator(prev_word_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, sequence_length), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        for i in range(batch_size):
            for t, w in enumerate(prev_word_list[index % len(prev_word_list)]):
                x[i, t] = word_index[w]
            y[i] = word_index[next_word_list[index % len(prev_word_list)]]
            index = index + 1
        yield x, y

# So now we did all the preprocessing of our data and now we will build our model. We will use LSTM model for this task.

Definition of the model

In [98]:
# build the model: a single LSTM
model = Sequential()

# Add Input Embedding Layer
model.add(Embedding(input_dim=nb_words, output_dim=100))
    
# Add Hidden Layer 1 - LSTM Layer
model.add(LSTM(128))
model.add(Dropout(0.2))
    
# Add Output Layer
model.add(Dense(nb_words, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         1704400   
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 17044)             2198676   
                                                                 
Total params: 4020324 (15.34 MB)
Trainable params: 4020324 (15.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Now we can train our model.

In [99]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(prev_words, next_words, test_size=0.2, random_state=42)

In [100]:
BATCH_SIZE = 32
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
model.fit(generator(X_train, y_train, BATCH_SIZE), steps_per_epoch = int(len(prev_words)/BATCH_SIZE) + 1, epochs=100, verbose=1, callbacks=[early_stopping],validation_data=generator(X_test, y_test, BATCH_SIZE), validation_steps=int(len(X_test)/BATCH_SIZE) + 1)

Epoch 1/100
2334/2334 [==============================] - 129s 54ms/step - loss: 6.7947 - accuracy: 0.1170 - val_loss: 6.7257 - val_accuracy: 0.1215
Epoch 2/100
2334/2334 [==============================] - 126s 54ms/step - loss: 6.0690 - accuracy: 0.1367 - val_loss: 6.6451 - val_accuracy: 0.1412
Epoch 3/100
2334/2334 [==============================] - 119s 51ms/step - loss: 5.4714 - accuracy: 0.1632 - val_loss: 6.6577 - val_accuracy: 0.1512
Epoch 4/100
2334/2334 [==============================] - 118s 51ms/step - loss: 4.9042 - accuracy: 0.1925 - val_loss: 6.6910 - val_accuracy: 0.1637
Epoch 5/100
2334/2334 [==============================] - 142s 61ms/step - loss: 4.3429 - accuracy: 0.2324 - val_loss: 6.9247 - val_accuracy: 0.1700
Epoch 6/100
2334/2334 [==============================] - 141s 60ms/step - loss: 3.7981 - accuracy: 0.2884 - val_loss: 6.9137 - val_accuracy: 0.1806
Epoch 7/100
2334/2334 [==============================] - 138s 59ms/step - loss: 3.2975 - accuracy: 0.3582 - val_

We save it in case we want only to reuse it as it is.

In [196]:
model.save("my_model.keras")


In [87]:
import keras
#model = keras.models.load_model("my_model.keras")

In [88]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1704400   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 17044)             2198676   
                                                                 
Total params: 4020324 (15.34 MB)
Trainable params: 4020324 (15.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Now that it's trained we can use it to generate lyrics.

Firslty we are just taking the best next word from the model.

In [236]:
source = "je t'aime a la folie"
source = source.split(' ')
n_token = 100
lyrics = source.copy()

In [237]:
source

['je', "t'aime", 'a', 'la', 'folie']

In [238]:
input_text = source.copy()

In [239]:
temperature = 1  # Adjust the temperature value for different levels of randomness

for i in range(n_token):
    encoded = []
    for word in input_text:
        encoded.append(word_index[word])
    encoded = np.array(encoded)
    encoded = encoded.reshape(1, len(encoded))
    y_pred = model.predict(encoded, verbose=0)
    y_pred = y_pred / temperature  
    y_pred = y_pred/np.sum(y_pred)
    # Apply temperature to adjust the randomness of predictions
    y_pred = np.random.choice(range(nb_words), size=1, p=y_pred[0])[0]
    input_text.append(index_word[y_pred])
    if len(input_text) > sequence_length:
        input_text = input_text[1:]
    lyrics.append(index_word[y_pred])
print(' '.join(lyrics))

je t'aime a la folie  
  dans l’dos mon rap poto te dire qu'il te "tu poussette j'porte pas l'temps, le temps  
  c'est pas facile, tu lui as tranquille j'en connais  
   
  et j'te vois pas j’observe  
  j'fume des petits avant parler de malade  
  il a grandi sur la rue me crois peu, s'énerve,  
  elle voir dans la suis dans le quartier  
  et on double on c’est la grosse tête  
  ça parle de danse, ouais  
  demande à azzedine on nous chez sur  
   
  j’fais pas mon temps là you might also like  
  aski' parait j'ai
